In [49]:
import joblib
import pandas as pd
import string
import re
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
import pickle

In [77]:
def svm_model():
    df = pd.read_csv('Review_Train2.csv')
    heshe = stopwords.words('english')
    nltk.download('stopwords')
    nltk.download('wordnet')
    replace_wo_space = re.compile("[.;:!\'?,\"()\[\]]")
    replace_with_space = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    reviews = df['review'].str.replace(replace_wo_space, '').str.lower()
    reviews = reviews.str.replace(replace_with_space, '')
    reviews_stopworded = reviews.apply(lambda x: ' '.join([word for word in str(x).split()
                                                           if word not in heshe]))
    lem = WordNetLemmatizer()
    reviews_lem = reviews_stopworded.apply(lambda x: ' '.join([lem.lemmatize(word) for word in str(x).split()]))
    ngram_cv = CountVectorizer(binary=True, ngram_range=(1, 2))
    ngram_cv.fit(reviews_lem)
    X = ngram_cv.transform(reviews_lem)
    target = df['Sentiment'].astype(str)
    X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=42)
    # I had to set the max_iter because it was not able to converge the SVM in the default 1,000 iterations
    grid_params = {'C': [0.25, 0.5, 0.75, 1.0]}
    svm = GridSearchCV(LinearSVC(multi_class='ovr', max_iter=1000000000, random_state=42), grid_params, cv=5)
    svm.fit(X_train, y_train)
    y_train_pred = svm.predict(X_train)
    print("Training accuracy is {}".format(accuracy_score(y_train, y_train_pred)))
    y_test_pred = svm.predict(X_test)
    print("Testing accuracy is {}".format(accuracy_score(y_test, y_test_pred)))
    # Save the vectorizer
    vec_file = 'vectorizer.pickle'
    pickle.dump(ngram_cv, open(vec_file, 'wb'))

    # Save the model
    mod_file = 'classification.model'
    pickle.dump(svm, open(mod_file, 'wb'))

In [78]:
def get_word_net_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [79]:
def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_word_net_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return text

In [80]:
def predict_review(review):
    # load the vectorizer
    loaded_vectorizer = pickle.load(open('vectorizer.pickle', 'rb'))
    # load the model
    loaded_model = pickle.load(open('classification.model', 'rb'))
    result = loaded_model.predict(loaded_vectorizer.transform([review]))
    new_result = str(result)[1:-1]
    sentiment = new_result.replace("'", "")
    return sentiment

In [83]:
svm_model()

[nltk_data] Downloading package stopwords to C:\Users\Muhammad
[nltk_data]     Waseem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Muhammad
[nltk_data]     Waseem\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Training accuracy is 0.9933504670990903
Testing accuracy is 0.9503997389459945


In [106]:
predict_review('I have edited three videos on this app so far and loved it until....I finished editing a video Feb 6th, uploaded it to YouTube, sent out the link to family and friends and it played fine. I went back to watch it this morning and my video clips within the video are missing. They played perfectly fine before. Very frustrating to put in so much work, everything be fine, and 2 days later my work is messed up.')

'Positive'